# Asignar género

- [ ] El tener multiples caracterisiticas demograficas permite hacer cortes y el sesgo...

El poder asignar un género a los tweets nos permite hacer un estudio de como los diferentes géneros abordan el tema de estudio, en este caso el de Migración. Twitter no recolecta esa información al hacer la cuenta, por lo que no podemos encontrar un atributo dado por la API, pero al igual que la geolocalización, podemos inferirla a partir de atributos como el nombre, la descripción o incluso de la foto de perfil.

En esta notebook se van a revisar algunas opciones para asignar género al autor:
- Buscar nombres típicamente masculinos o femeninos en los campos de username, descripción
- Buscar pronombres (She/Her o He/His) en la descripción del usuario
- Buscar la nacionalidad (e.g.: Mexicano / Mexicana) en los campos descripción

También se podría trabajar con adjetivos, profesiones, oficios o títulos.

Nota: Dado que estos campos son abiertos y el nivel de veracidad recae completamente en cada usuario, se puede esperar que estos métodos tengan falsos positivos o negativos. Igualmente, tampoco se puede esperar que se termine asignando un género a todos los autores.

In [1]:
import os
import re
import pandas as pd

import helpers.preprocess_text as pre_text

## Cargar nombres, pronombres, nacionalidades y géneros

En el directorio llamado ./resources hay un archivo llamado names_genders.csv que contiene una columna con nombres, una con el género que típicamente tienen las personas llamadas así y una última columna de si es un nombre compuesto como; Maria Jose o Jose Guadalupe.

In [2]:
df_names = pd.read_csv('./resources/names_genders.csv')

pre_text.remove_duplicated_chars(df_names, 'name')

# Names composed
df_names_double = df_names[df_names['is_double']]
regex_names_double = f"\\b((?:{'|'.join(df_names_double['name'])}))\\b"
regex_names_double = re.compile(regex_names_double)

# Single names
df_names_single = df_names[~df_names['is_double']]
regex_names_single = f"\\b((?:{'|'.join(df_names_single['name'])}))\\b"
regex_names_single = re.compile(regex_names_single)

# Names to gender
dict_names_gender = df_names.set_index('name')['gender'].to_dict()
df_names

,name,gender,is_double
0,guadalupe,F,False
1,jose,M,False
2,sofia,F,False
3,angel,M,False
4,jesus,M,False
...,...,...,...
555,aron,M,False
556,jose guadalupe,M,True
557,jose maria,M,True
558,maria guadalupe,F,True


Despues hay un diccionario con los posibles strings conteniendo pronombres y el genero que identifican. 

In [3]:
dict_pronouns_gender = {
    "she her": 'F',
    "her she": 'F',
    "he him": 'M',
    "him he": 'M'
}

regex_pronous = f"\\b((?:{'|'.join(dict_pronouns_gender.keys())}))\\b"
regex_pronous = re.compile(regex_pronous)
regex_pronous

re.compile(r'\b((?:she her|her she|he him|him he))\b', re.UNICODE)

Un diccionario con las posibles nacionalidades en las diferentes variaciones segun el genero

In [4]:
dict_nationalities_gender = {
    'mexicano': 'M',
    'mexicana': 'F',
    'colombiano': 'M',
    'colombiana': 'F',
    'argentino': 'M',
    'argentina': 'F'
}

regex_nationalities = f"\\b((?:{'|'.join(dict_nationalities_gender.keys())}))\\b"
regex_nationalities = re.compile(regex_nationalities)
regex_nationalities

re.compile(r'\b((?:mexicano|mexicana|colombiano|colombiana|argentino|argentina))\b',
re.UNICODE)

## Cargar tweets

Primero hay que cargar los tweets, específicamente el ID del tweet, el ID del usuario, el nombre del autor, su descripción. Los últimos tres campos son abiertos, es decir, que el usuario puede ingresar lo que desee a excepción de algunos caracteres especiales. Esto implica que antes de todo hay que pre-procesarlos para así reducir el número de variantes para una misma nacionalidad y/o nombre.

Nota: Usaremos de ejemplo el primer dataset que se obtuvo en la notebook "2_Definiendo_Queries.ipynb".

In [5]:
df_tweets = pd.read_json("./archivos_queries/1_dataset.jsonl", lines=True)
df_tweets = df_tweets[['id', 'author_id', 'author']]

df_tweets['author_name'] = df_tweets['author'].apply(lambda x: x['name'])
df_tweets['author_description'] = df_tweets['author'].apply(lambda x: x['description'])

df_tweets = df_tweets.drop(columns=['author'])

df_tweets

,id,author_id,author_name,author_description
0,1565119364566847488,1423647858650136576,Subsecretaría de Gobierno,MUNICIPALIDAD DE SAN MIGUEL - BUENOS AIRES
1,1565104878166446080,195947504,Personería Distrital de Medellín,#PorTusDerechosMásCerca #SomosPersonería #Pers...
2,1565103949337837568,286819842,Simón Gamboa,activista Colombovenezolano🇻🇪🇨🇴 |concejal de C...
3,1565102572586909696,1284595923754999808,Cesar Cortes,"100% neoliberal, 100% aspiracionista, 100% mex..."
4,1565095116074876928,1333482129716604928,Mauro Grande,I'm stuck in a city but I belong in a field.\n...
...,...,...,...,...
8579,1477088691877261312,397706715,FUVADIS INTERNACIONAL 🏳️‍🌈,"Trabajamos por la inclusión, el respeto y la i..."
8580,1477088689901682688,397706715,FUVADIS INTERNACIONAL 🏳️‍🌈,"Trabajamos por la inclusión, el respeto y la i..."
8581,1477086472469684224,1176580744036196352,GaelAlejandro,¡Estoy vivooo!
8582,1477067319830478848,579360940,Juan Farre,#11


Para evitar trabajar con autores duplicados, trabajaremos solo con una copia única de los autores, asignaremos géneros y luego haremos JOIN a los tweets usando como llave la columna `author_id`.

In [6]:
df_users = df_tweets.drop_duplicates(subset=['author_id'], 
                                  keep='last').reset_index(drop=True)
df_users = df_users.iloc[:, 1:]
df_users

,author_id,author_name,author_description
0,1423647858650136576,Subsecretaría de Gobierno,MUNICIPALIDAD DE SAN MIGUEL - BUENOS AIRES
1,1333482129716604928,Mauro Grande,I'm stuck in a city but I belong in a field.\n...
2,492397405,Magdalena Ayala,"Hay que hacerse cargo de lo que uno dice, fin."
3,132020962,Johanna Saenz,Mamá. Politóloga. Ms en Estudios Latinoamerica...
4,4106090319,〽️☕,1997.\n• Club Atlético River Plate🐓❤
...,...,...,...
4524,377788360,Eduardo Díatri🤍💙❤️☮️💛💙,Productor y asociado para espectáculos mayores...
4525,397706715,FUVADIS INTERNACIONAL 🏳️‍🌈,"Trabajamos por la inclusión, el respeto y la i..."
4526,1176580744036196352,GaelAlejandro,¡Estoy vivooo!
4527,579360940,Juan Farre,#11


## Preprocesamiento

La siguiente función se encarga de pasar todo a minúsculas, eliminar acentos, URLs, emails, números, caracteres que no sean letras y espacios en blancos duplicados. Esto nos ayudará a que al buscar los nombres o nacionalidades en los campos, se tenga más posibilidades de encontrarlos, dado que no habrá variaciones del tipo: "NOE", "NoE", "Noe", "Noé". Permitiendo que todas esas variaciones sean validas y encontradas con la cadena de texto "noe".

In [7]:
def preprocess_for_names(df_data, new_col, flg_remove_two_chars=True):

    at_least_3_letter_reg = re.compile(r'\b\w{1,2}\b')
    at_least_3_letter_reg_capture = re.compile(r'\b(\w{2})\b')

    pre_text.initial_preprocessing(df_data, new_col,
                                   flg_remove_emojis=True,
                                   flg_lower=True)

    pre_text.remove_urls(df_data, new_col)
    pre_text.remove_emails(df_data, new_col)
    pre_text.remove_urls(df_data, new_col)
    pre_text.remove_numbers(df_data, new_col, 
                            replace_char=' ')

    pre_text.remove_duplicated_chars(df_data, new_col)
    pre_text.remove_non_alphanumeric(df_data, new_col)

    pre_text.remove_single_letters(df_data, new_col)
    
    if(flg_remove_two_chars):
        df_data[new_col] = df_data[new_col].str.replace(at_least_3_letter_reg, 
                                                        ' ', regex=True)

    pre_text.remove_multiple_blank_spaces(df_data, new_col)
    df_data[new_col] = df_data[new_col].str.strip()

    valid_texts = ((~df_data[new_col].isna()) *
                   (df_data[new_col] != '') *
                   (~df_data[new_col].str.contains(r'^\w$', regex=True)))
    df_data = df_data[valid_texts]

    return df_data

Se le aplica la función de limpieza a ambas columnas

In [8]:
df_users = preprocess_for_names(df_users.copy(), 'author_name')
df_users = preprocess_for_names(df_users.copy(), 'author_description',
                                flg_remove_two_chars=False)
df_users

,author_id,author_name,author_description
0,1423647858650136576,subsecretaria gobierno,municipalidad de san miguel buenos aires
1,1333482129716604928,mauro grande,stuck in city but belong in field recibido en ...
2,492397405,magdalena ayala,hay que hacerse cargo de lo que uno dice fin
3,132020962,johana saenz,mama politologa ms en estudios latinoamericano...
5,809515814785347584,daniel vitolo,abogado docente consejero en uba derecho por f...
...,...,...,...
4523,452492064,alfredo marquez,caminando su lado refugio de peludos rescatado...
4524,377788360,eduardo diatri,productor asociado para espectaculos mayores d...
4525,397706715,fuvadis internacional,trabajamos por la inclusion el respeto la inte...
4526,1176580744036196352,gaelalejandro,estoy vivo


## Buscar los nombres en la columna del nombre

In [ ]:
# Copia de la tabla de usuarios
df_by_names = preprocess_for_names(df_users.copy(), 'author_name')

# Buscar nombres
df_tmp2 = df_by_names['author_name'].str.extract(regex_names_double).dropna()

df_tmp1 = df_by_names[~df_by_names.index.isin(df_tmp2.index)]['author_name']
df_tmp1 = df_tmp1.str.extract(regex_names_single).dropna()

# Asignar género en base al nombre encontrado
df_by_names['extracted_name'] = pd.concat([df_tmp1, df_tmp2])[0]
df_by_names['gender'] = df_by_names['extracted_name'].map(dict_names_gender)

df_by_names = df_by_names.dropna()
df_by_names[['author_name', 'extracted_name', 'gender']]

## Buscar los probombres en la descripción

In [10]:
# Descartar usuarios a los que ya se les asigno un género
df_pronouns = df_users[~df_users['author_id'].isin(df_by_names['author_id'])].copy()

# Buscar los probombres en la descripción
df_pronouns['extracted_pronoun'] = df_pronouns['author_description'].str.extract(regex_pronous).dropna()

# Asignar un género en base a los pronombres encontrados
df_pronouns['gender'] = df_pronouns.dropna()['extracted_pronoun'].map(dict_pronouns_gender)

df_pronouns = df_pronouns.dropna()
df_pronouns[['author_name', 'extracted_name', 'gender']]

,author_id,author_name,author_description,extracted_pronoun,gender
359,149504801,celia mendoza,she her ela journalist colombian american pasi...,she her,F
588,1244223031230291968,vino barato,el he him biologo humano semi profesional otak...,he him,M
1465,1202200529058836480,gabe itch,nie chce odwiedzac juz fikcyjnych miejsc posta...,he him,M
1522,53093155,karol suarez,journalist based out of mexico city for latam ...,she her,F
1550,1092160641987686400,edecan telcel taylor version,he him write love leter to yourself,he him,M
1621,860704940691529728,lencha huasteca uta stan,trans and proud\nshe her they them digital art...,she her,F
2065,46689241,pistolita,mexican gender fluid she her ela brand strateg...,she her,F
2212,26028712,alain rodriguez castro,journalist at heart world traveler by choice h...,he him,M
2560,30244057,paz,alguien por aca me dijo twiterete she her and ...,she her,F
3033,259579243,paxton hernandez,golden kimbal is my role model mormon he him d...,he him,M


### Nota
Los siguientes métodos necesitan un mayor preprocesamiento del realizado en esta notebook, debido a que puede haber nombres de países que se confundan con la nacionalidad (e.g. Argentina) o los usuarios sean de una organización y/o negocio, pero mencionen a alguna nacionalidad. Se debería considerar añadir un paso de preprocesamiento donde se intente identificar que cuentas no son controladas por un solo individuo y representan a un conjunto.

## Buscar nacionalidades en la descripción

In [11]:
# Descartar usuarios a los que ya se les asigno un género
df_nationalities = df_users.loc[~(df_users['author_id'].isin(df_by_names['author_id']) + 
                                  df_users['author_id'].isin(df_pronouns['author_id']))].copy()

# Buscar los probombres en la descripción
df_nationalities['extracted_nationality'] = df_nationalities['author_description'].str.extract(regex_nationalities).dropna()

# # Asignar un género en base a los pronombres encontrados
df_nationalities['gender'] = df_nationalities.dropna()['extracted_nationality'].map(dict_nationalities_gender)

df_nationalities = df_nationalities.dropna()
df_nationalities

,author_id,author_name,author_description,extracted_nationality,gender
94,333538162,meg,colombiana viviendo en peru feminista marea ve...,colombiana,F
129,884394090456129536,nota,genero derechos humanos desde el norte argenti...,argentino,M
290,1354452734116843520,pajareando aves naturaleza mexico,explorando conociendo la biodiversidad mexican...,mexicana,F
305,138795997,santi davio,ex jugador venezuela guatemala el salvador ita...,argentino,M
411,1347153765615468544,vzlametalfest,festival creado para promover las agrupaciones...,argentina,F
...,...,...,...,...,...
4347,1166728119434301440,secretaria trabajadores migrantes refugiad utep,trabajamos para defender los derechos de las l...,argentina,F
4413,317321263,raphael,colombiano democrata amante de la naturaleza,colombiano,M
4416,1402075690019786752,nanda astral,astrologa poetisa viajera colombiana amo los a...,colombiana,F
4417,1146517522927280128,hugok,tucuman argentina dios patria familia lic admi...,argentina,F


## Buscar nombres en la descripción del usuario

In [12]:
# Descartar usuarios a los que ya se les asigno un género
df_description = df_users.loc[~(df_users['author_id'].isin(df_by_names['author_id']) + 
                                df_users['author_id'].isin(df_pronouns['author_id']) + 
                                df_users['author_id'].isin(df_nationalities['author_id']))].copy()

# Extraer nombres de la descripción
df_tmp2 = df_description['author_description'].str.extract(regex_names_double).dropna()

df_tmp1 = df_description[~df_description.index.isin(df_tmp2.index)]['author_description']
df_tmp1 = df_tmp1.str.extract(regex_names_single).dropna()

# Asignar genero a los nombres
df_description['extracted_name'] = pd.concat([df_tmp1, df_tmp2])[0]
df_description['gender'] = df_description['extracted_name'].map(dict_names_gender)

df_description = df_description.dropna()
df_description

,author_id,author_name,author_description,extracted_name,gender
0,1423647858650136576,subsecretaria gobierno,municipalidad de san miguel buenos aires,miguel,M
28,2208251462,nacho marquez,papa de joaquin nicole melomano empleado amant...,joaquin,M
55,704525000049565696,gestion social,laura siera primera gestora karen estrada jefa...,laura,F
79,900998018,jorgematera,granate hasta los huesos indio solo pido mas p...,matias,M
92,4924501120,caminos areco,guia de turismo en san antonio de areco guia d...,antonio,M
...,...,...,...,...,...
4376,169302951,florencia carignano,mama de martu emi nacho mg salamanca especiali...,andres,M
4406,1327556625016688640,konsomol charki,nuestra vida no es digna de ser vivida sino cu...,jose,M
4467,183117973,lux lancheros,bebo se cosas miranda priestly del sitp,miranda,F
4475,65079478,esneyder negrete,periodista actualmente en efenoticias como edi...,america,F


## Más ideas
- Buscar profesiones y sus abstracciones en el nombre del la cuenta. E.g.: Doctor/Dr. y Doctora/Dra.
- Buscar titulos como "Papá", "Mamá", "Coordinador", "Coordinadora"
- Buscar adjetivos calificativos en la descripción
- Identificar cuentas que representen a Secretarias Publicas, Oraganización, Clubs, Empresas, etc.
- Aplicar un NER para hacer las busquedas de manera más inteligentes y focalizada

## Añadir la columna gender a cada tweet

Primero se juntan las tablas de los usuarios con género obtenidos con los diferentes métodos

In [13]:
df_genders = pd.concat([df_by_names, df_pronouns, df_nationalities, df_description])
df_genders = df_genders.drop(columns=['extracted_name', 'extracted_pronoun', ])

df_genders = df_genders[['author_id', 'gender']]
df_genders.columns = ['author_id', 'author_gender']
df_genders

,author_id,author_gender
1,1333482129716604928,M
2,492397405,F
3,132020962,F
5,809515814785347584,M
8,97038156,M
...,...,...
4376,169302951,M
4406,1327556625016688640,M
4467,183117973,F
4475,65079478,F


Se hace el merge de las tablas, el equivalente a un JOIN de SQL

In [14]:
df_tweets = df_tweets.merge(df_genders, on='author_id', how='left')
df_tweets

,id,author_id,author_name,author_description,author_gender
0,1565119364566847488,1423647858650136576,Subsecretaría de Gobierno,MUNICIPALIDAD DE SAN MIGUEL - BUENOS AIRES,M
1,1565104878166446080,195947504,Personería Distrital de Medellín,#PorTusDerechosMásCerca #SomosPersonería #Pers...,NaN
2,1565103949337837568,286819842,Simón Gamboa,activista Colombovenezolano🇻🇪🇨🇴 |concejal de C...,NaN
3,1565102572586909696,1284595923754999808,Cesar Cortes,"100% neoliberal, 100% aspiracionista, 100% mex...",M
4,1565095116074876928,1333482129716604928,Mauro Grande,I'm stuck in a city but I belong in a field.\n...,M
...,...,...,...,...,...
8579,1477088691877261312,397706715,FUVADIS INTERNACIONAL 🏳️‍🌈,"Trabajamos por la inclusión, el respeto y la i...",NaN
8580,1477088689901682688,397706715,FUVADIS INTERNACIONAL 🏳️‍🌈,"Trabajamos por la inclusión, el respeto y la i...",NaN
8581,1477086472469684224,1176580744036196352,GaelAlejandro,¡Estoy vivooo!,NaN
8582,1477067319830478848,579360940,Juan Farre,#11,NaN


TODOs
- [ ] Añadir una grafica donde compare cuantos M y F publicaron para cada pais
- [ ] Otra grafica donde solo sea un pie o barras que compare M, F y apagado aquellos con NaN